### Load Dataset

In [3]:
from datasets import load_dataset

dataset = load_dataset("Anthropic/hh-rlhf", data_dir="helpful-base", split='train')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

### Save raw dataset

In [17]:
dataset.to_json('question_raw.jsonl')

import pandas as pd    
jsonObj = pd.read_json(path_or_buf='question_raw.jsonl', lines=True)
jsonObj

Creating json from Arrow format:   0%|          | 0/44 [00:00<?, ?ba/s]

,chosen,rejected
0,"\n\nHuman: Hi, I want to learn to play horsesh...","\n\nHuman: Hi, I want to learn to play horsesh..."
1,\n\nHuman: How do I teach kids to meditate?\n\...,\n\nHuman: How do I teach kids to meditate?\n\...
2,\n\nHuman: Can you tell me the steps for getti...,\n\nHuman: Can you tell me the steps for getti...
3,\n\nHuman: How can I store food if I don't hav...,\n\nHuman: How can I store food if I don't hav...
4,\n\nHuman: what are some good novels for a 9 y...,\n\nHuman: what are some good novels for a 9 y...
...,...,...
43830,\n\nHuman: I'm planning to make cookies for Ch...,\n\nHuman: I'm planning to make cookies for Ch...
43831,\n\nHuman: What ingredients do I need to make ...,\n\nHuman: What ingredients do I need to make ...
43832,\n\nHuman: Can you find a guide for caring for...,\n\nHuman: Can you find a guide for caring for...
43833,\n\nHuman: I like to host guests at my home fr...,\n\nHuman: I like to host guests at my home fr...


### Process and Save Dataset

In [18]:
import re

jsonObj.chosen = jsonObj.chosen.apply(lambda st: st.split('\n\n')[1]).str.replace('Human: ', '').apply(lambda st: [st])
jsonObj = jsonObj.reset_index().drop("rejected", axis=1)
jsonObj.rename(columns = {'index':'question_id', 'chosen': 'turns'}, inplace = True) 
jsonObj.to_json('question_raw.jsonl', orient='records', lines=True)
jsonObj

None


,question_id,turns
0,0,"[Hi, I want to learn to play horseshoes. Can y..."
1,1,[How do I teach kids to meditate?]
2,2,[Can you tell me the steps for getting a harbo...
3,3,[How can I store food if I don't have a pantry?]
4,4,[what are some good novels for a 9 year old?]
...,...,...
43830,43830,[I'm planning to make cookies for Christmas. ...
43831,43831,[What ingredients do I need to make a margheri...
43832,43832,[Can you find a guide for caring for a new tat...
43833,43833,[I like to host guests at my home from time to...


We then filter the `question_raw.jsonl` file to create `question.jsonl` using the GPT-4 prompt:

https://chat.openai.com/share/c8d01867-f556-4d60-9a4c-6428fd722376